In [26]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder,FunctionTransformer,Normalizer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import numpy as np
from imblearn.pipeline import Pipeline 
from sklearn.metrics import classification_report, confusion_matrix,precision_score, recall_score,f1_score,accuracy_score

In [27]:
import pandas as pd


In [28]:
df = pd.read_csv("/Users/Documents/ML_OPS/DAY9_end2end/dataset/fitness_dataset.csv")
df

,age,height_cm,weight_kg,heart_rate,blood_pressure,sleep_hours,nutrition_quality,activity_index,smokes,gender,is_fit
0,56,152,65,69.6,117.0,NaN,2.37,3.97,no,F,1
1,69,186,95,60.8,114.8,7.5,8.77,3.19,0,F,1
2,46,192,103,61.4,116.4,NaN,8.20,2.03,0,F,0
3,32,189,83,60.2,130.1,7.0,6.18,3.68,0,M,1
4,60,175,99,58.1,115.8,8.0,9.95,4.83,yes,F,1
...,...,...,...,...,...,...,...,...,...,...,...
1995,52,173,98,60.7,106.1,NaN,1.54,3.25,1,M,1
1996,61,186,74,51.4,123.8,9.4,8.63,3.15,no,M,1
1997,77,198,89,76.7,103.6,8.3,1.98,3.36,yes,M,0
1998,62,190,63,80.7,115.9,6.7,9.21,2.39,1,F,0


In [29]:
df.isnull().sum()

age                    0
height_cm              0
weight_kg              0
heart_rate             0
blood_pressure         0
sleep_hours          160
nutrition_quality      0
activity_index         0
smokes                 0
gender                 0
is_fit                 0
dtype: int64

In [30]:
df.duplicated().sum()

0

In [31]:
df['BMI'] = df['weight_kg'] / ((df['height_cm'] / 100) ** 2)
df.head(5)

,age,height_cm,weight_kg,heart_rate,blood_pressure,sleep_hours,nutrition_quality,activity_index,smokes,gender,is_fit,BMI
0,56,152,65,69.6,117.0,NaN,2.37,3.97,no,F,1,28.133657
1,69,186,95,60.8,114.8,7.5,8.77,3.19,0,F,1,27.459822
2,46,192,103,61.4,116.4,NaN,8.20,2.03,0,F,0,27.940538
3,32,189,83,60.2,130.1,7.0,6.18,3.68,0,M,1,23.235632
4,60,175,99,58.1,115.8,8.0,9.95,4.83,yes,F,1,32.326531


In [32]:
df['smokes'] = df['smokes'].replace({
    'yes': 1, 'no': 0,
    'Yes': 1, 'No': 0,
    'Y': 1, 'N': 0,
    True: 1, False: 0
})
df['smokes'] = df['smokes'].astype(int)


In [33]:
df['gender'] = df['gender'].replace({
    'male': 1, 'female': 0,
    'Male': 1, 'female': 0,
    'M': 1, 'F': 0,
    'm': 1, 'f': 0
})
df['gender'] = df['gender'].astype(int)

In [34]:
df["sleep_hours"].fillna(df["sleep_hours"].mean(),inplace=True)

In [35]:
#lấy dữ liệu đầu vào và đầu ra
target = "is_fit"
x =  df.drop(target, axis=1)
y = df[target]
x.shape, y.shape
y.value_counts()
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.2,stratify = y,random_state=42)
print("Train data shape:", x_train.shape)
print("Test data shape:", x_test.shape)
print("y_train:")
y_test.value_counts()



Train data shape: (1600, 11)
Test data shape: (400, 11)
y_train:


is_fit
0    240
1    160
Name: count, dtype: int64

In [36]:
num_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
nom_transformer = Pipeline(steps=[
    ("one", OneHotEncoder())
])
pre_processing3  = ColumnTransformer(
    transformers=[
       ("num", num_transformer, ['age', 'heart_rate', 'blood_pressure',
        'nutrition_quality', 'activity_index','sleep_hours','smokes','BMI','height_cm','weight_kg','gender']),
    ]
)

In [38]:
model_svc = Pipeline(steps=[
    ('preprocessing', pre_processing3),
    ('clf', SVC(C = 10, gamma=0.01, kernel='linear', random_state=42))
])
model_svc.fit(x_train, y_train)
y_pred = model_svc.predict(x_test)
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred, average='macro')}")
print(f"F1 Score: {f1_score(y_test, y_pred, average='macro')}")
print(f"Recall: {recall_score(y_test, y_pred, average='macro')}")
print(confusion_matrix(y_test, y_pred))
for i,j in zip(y_test[:100], y_pred[:100]):
    print(f"Actual: {i}, Predicted: {j}")

              precision    recall  f1-score   support

           0       0.80      0.87      0.83       240
           1       0.77      0.68      0.72       160

    accuracy                           0.79       400
   macro avg       0.79      0.77      0.78       400
weighted avg       0.79      0.79      0.79       400

Accuracy: 0.7925
Precision: 0.7880692242394369
F1 Score: 0.7789599131818454
Recall: 0.7739583333333333
[[208  32]
 [ 51 109]]
Actual: 1, Predicted: 1
Actual: 1, Predicted: 1
Actual: 0, Predicted: 0
Actual: 0, Predicted: 0
Actual: 0, Predicted: 0
Actual: 0, Predicted: 0
Actual: 0, Predicted: 0
Actual: 0, Predicted: 0
Actual: 0, Predicted: 1
Actual: 1, Predicted: 1
Actual: 1, Predicted: 0
Actual: 0, Predicted: 0
Actual: 0, Predicted: 0
Actual: 1, Predicted: 1
Actual: 1, Predicted: 1
Actual: 0, Predicted: 0
Actual: 0, Predicted: 0
Actual: 0, Predicted: 0
Actual: 0, Predicted: 0
Actual: 1, Predicted: 0
Actual: 0, Predicted: 1
Actual: 1, Predicted: 0
Actual: 0, Predicte